In [1]:
import tushare as ts
import pandas as pd

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [3]:
concept = pro.concept()

In [4]:
len(concept)

359

In [5]:
concept.head()

,code,name,src
0,TS0,密集调研,ts
1,TS1,南北船合并,ts
2,TS2,5G,ts
3,TS3,机场,ts
4,TS4,高价股,ts


In [6]:
concept = concept.drop(columns=['src'])
concept.columns = ['concept_code:ID(Concept)', 'concept_name:string']
concept[':LABEL'] = 'Concept'

In [7]:
len(concept)

359

In [8]:
concept.head()

,concept_code:ID(Concept),concept_name:string,:LABEL
0,TS0,密集调研,Concept
1,TS1,南北船合并,Concept
2,TS2,5G,Concept
3,TS3,机场,Concept
4,TS4,高价股,Concept


In [9]:
pro.concept_detail(id = 'TS0', fields='id,ts_code').head()

,id,ts_code
0,TS0,000301.SZ
1,TS0,000401.SZ
2,TS0,000932.SZ
3,TS0,002013.SZ
4,TS0,002106.SZ


In [11]:
import time

company_concept = []
for i, row in concept.iterrows():
    concept_code = row['concept_code:ID(Concept)']
    concept2company = pro.concept_detail(id = concept_code, fields='id,ts_code')
    company_concept.append(concept2company)
    time.sleep(0.5)

company_concept = pd.concat(company_concept)

In [12]:
company_concept.columns = [':END_ID(Concept)', ':START_ID(Company)']
company_concept[':TYPE'] = 'in_concept'

In [13]:
companys = pro.stock_basic(exchange='', list_status='L', fields='ts_code')
companys = set(companys['ts_code'])

In [14]:
m = company_concept.apply(lambda x: x[':START_ID(Company)'] in companys, axis=1)

In [15]:
company_concept = company_concept[m]

In [16]:
company_concept.head()

,:END_ID(Concept),:START_ID(Company),:TYPE
0,TS0,000301.SZ,in_concept
1,TS0,000401.SZ,in_concept
2,TS0,000932.SZ,in_concept
3,TS0,002013.SZ,in_concept
4,TS0,002106.SZ,in_concept


In [17]:
concept.head()

,concept_code:ID(Concept),concept_name:string,:LABEL
0,TS0,密集调研,Concept
1,TS1,南北船合并,Concept
2,TS2,5G,Concept
3,TS3,机场,Concept
4,TS4,高价股,Concept


In [18]:
concept.to_csv('import/concept.csv', index=False)
company_concept.to_csv('import/company_concept.csv', index=False)